### This is the code for extraction images form each subdirectory and evaluating the model
#### A similar code was used for creating the numpy array used in the main training code

In [1]:
import PIL
from PIL import Image
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dense, Dropout, Activation, Flatten
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from keras import regularizers

from keras.models import model_from_json
from keras.utils import to_categorical
from keras.preprocessing import image

import matplotlib.pyplot as plt

from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.layers import Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.initializers import glorot_normal
from keras.utils import np_utils
from keras.metrics import top_k_categorical_accuracy
import tensorflow as tf

Using TensorFlow backend.
/home/kousik/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kousik/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kousik/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kousik/.local/lib/python3.6/site-packages/tensorflow/python

In [6]:
def evaluate(model_path,dir_path):
    folders=['tilia_americana', 'fraxinus_americana', 'abies_nordmanniana', 'pinus_rigida', 'celtis_tenuifolia', 'betula_alleghaniensis', 'acer_ginnala', 'liriodendron_tulipifera', 'ulmus_americana', 'pinus_koraiensis', 'catalpa_speciosa', 'pinus_sylvestris', 'quercus_bicolor', 'aesculus_hippocastamon', 'cryptomeria_japonica', 'morus_rubra', 'pinus_densiflora', 'larix_decidua', 'chamaecyparis_pisifera', 'pinus_parviflora', 'chionanthus_retusus', 'picea_orientalis', 'carya_cordiformis', 'quercus_velutina', 'malus_coronaria', 'quercus_phellos', 'eucommia_ulmoides', 'salix_babylonica', 'betula_nigra', 'acer_palmatum', 'prunus_yedoensis', 'quercus_imbricaria', 'aesculus_flava', 'magnolia_virginiana', 'cedrus_deodara', 'salix_matsudana', 'sassafras_albidum', 'magnolia_tripetala', 'magnolia_denudata', 'ginkgo_biloba', 'fagus_grandifolia', 'quercus_palustris', 'pinus_strobus', 'magnolia_macrophylla', 'ostrya_virginiana', 'tilia_europaea', 'betula_populifolia', 'picea_abies', 'pinus_bungeana', 'amelanchier_laevis', 'magnolia_stellata', 'pseudolarix_amabilis', 'quercus_marilandica', 'acer_pensylvanicum', 'acer_griseum', 'tilia_tomentosa', 'liquidambar_styraciflua', 'juglans_nigra', 'populus_deltoides', 'pinus_resinosa', 'prunus_serrulata', 'acer_negundo', 'carpinus_betulus', 'quercus_shumardii', 'fraxinus_pennsylvanica', 'zelkova_serrata', 'quercus_montana', 'pinus_taeda', 'evodia_daniellii', 'koelreuteria_paniculata', 'cedrus_atlantica', 'populus_grandidentata', 'acer_saccharum', 'salix_caroliniana', 'juglans_cinerea', 'carya_ovata', 'amelanchier_canadensis', 'platanus_acerifolia', 'paulownia_tomentosa', 'populus_tremuloides', 'ulmus_glabra', 'castanea_dentata', 'quercus_robur', 'syringa_reticulata', 'tilia_cordata', 'celtis_occidentalis', 'oxydendrum_arboreum', 'gleditsia_triacanthos', 'quercus_falcata', 'cornus_mas', 'toona_sinensis', 'chionanthus_virginicus', 'malus_hupehensis', 'tsuga_canadensis', 'prunus_virginiana', 'carya_tomentosa', 'picea_pungens', 'ilex_opaca', 'carya_glabra', 'pinus_thunbergii', 'quercus_alba', 'magnolia_acuminata', 'quercus_macrocarpa', 'pinus_peucea', 'magnolia_grandiflora', 'styrax_obassia', 'quercus_coccinea', 'carpinus_caroliniana', 'gymnocladus_dioicus', 'cornus_kousa', 'quercus_rubra', 'pinus_flexilis', 'aesculus_glabra', 'juniperus_virginiana', 'chamaecyparis_thyoides', 'salix_nigra', 'ulmus_parvifolia', 'acer_saccharinum', 'crataegus_viridis', 'prunus_sargentii', 'diospyros_virginiana', 'pyrus_calleryana', 'cedrus_libani', 'crataegus_laevigata', 'styrax_japonica', 'ulmus_procera', 'malus_angustifolia', 'prunus_pensylvanica', 'crataegus_crus-galli', 'cercis_canadensis', 'malus_pumila', 'pinus_cembra', 'acer_pseudoplatanus', 'broussonettia_papyrifera', 'amelanchier_arborea', 'maclura_pomifera', 'quercus_cerris', 'fraxinus_nigra', 'pinus_virginiana', 'nyssa_sylvatica', 'pinus_pungens', 'malus_baccata', 'magnolia_soulangiana', 'acer_rubrum', 'robinia_pseudo-acacia', 'asimina_triloba', 'cercidiphyllum_japonicum', 'betula_jacqemontii', 'metasequoia_glyptostroboides', 'ficus_carica', 'abies_concolor', 'pinus_wallichiana', 'quercus_nigra', 'quercus_michauxii', 'albizia_julibrissin', 'acer_campestre', 'quercus_muehlenbergii', 'platanus_occidentalis', 'quercus_virginiana', 'morus_alba', 'crataegus_pruinosa', 'taxodium_distichum', 'staphylea_trifolia', 'crataegus_phaenopyrum', 'pinus_nigra', 'betula_lenta', 'prunus_subhirtella', 'prunus_serotina', 'aesculus_pavi', 'ulmus_pumila', 'cornus_florida', 'ulmus_rubra', 'malus_floribunda', 'phellodendron_amurense', 'acer_platanoides', 'quercus_stellata', 'ailanthus_altissima', 'corylus_colurna', 'quercus_acutissima', 'ptelea_trifoliata', 'cladrastis_lutea', 'pinus_echinata', 'stewartia_pseudocamellia', 'halesia_tetraptera', 'catalpa_bignonioides']
    folders.sort()
    X=[]
    file_names=[]
    y=[]
    size=(64,64)
    for j in tqdm(range(len(folders))):
        for (root,dirs,files) in os.walk(dir_path+folders[j]): 

            for i in files:
                img=Image.open(dir_path+folders[j]+'/'+i)
                img = img.resize(size, PIL.Image.ANTIALIAS)
                X.append(np.asarray(img)/255.)
                y.append(folders[j])
                file_names.append(i)
    X=np.array(X)
    labels_df=pd.DataFrame({"files":file_names,"labels":y})
    y_oh=np.array(pd.get_dummies(y))
    model=load_model(model_path)
    predictions = model.predict(X)

    output = tf.keras.metrics.top_k_categorical_accuracy(y_oh, predictions, k=3)

    with tf.Session() as sess:
        result = sess.run(output)

    
    
    print("Top 1 accuracy", model.evaluate(X,y_oh)[1])
    print("Top 3 accuracy", result.sum()/len(result))
    
    
    

In [7]:
evaluate("./model.h5","../leaf_classification/dataset/test/")

100%|██████████| 185/185 [00:06<00:00, 27.80it/s]




3090/3090 [==============================] - 4s 1ms/step
Top 1 accuracy 0.9809061288833618
Top 3 accuracy 0.998705501618123


# Dataset path should contain '/' at end like "./dataset/test/"